# Imports and Data Directory

In [1]:
# from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
import os
import pandas as pd
import numpy as np
import shutil
import zipfile

# Delete data directory if present.
shutil.rmtree('ECEN489Py4Data', ignore_errors=True)

# Extract data.
with zipfile.ZipFile("ECEN489Py4Data.zip", "r") as zip_ref:
    zip_ref.extractall('ECEN489Py4Data')

with zipfile.ZipFile("ECEN489Py4TestData.zip", "r") as zip_ref:
    zip_ref.extractall(os.path.join("ECEN489Py4Data/data"))
    
# Set directory for data.
DATA_DIR = os.path.join('ECEN489Py4Data', 'data')

Using TensorFlow backend.


# Image Data Generation

In [2]:
#ImageDataGenerator implements functions useful for input image scaling and augmentation -- you may want more!

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


# Functions

In [3]:
def init_gen(image_dir, dg):
    """Initialize a flow_from_directory generator.
    
    We have to do this EACH TIME because otherwise we
    get different results do to some sort of shifting
    issue. 
    """
    if image_dir == 'train':
        shuffle = True
    elif image_dir == 'validation' or image_dir == 'test':
        shuffle = False
    else:
        raise UserWarning('Typo!')
        
    g = dg.flow_from_directory(
        os.path.join(DATA_DIR, image_dir),
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=shuffle,
        seed=1953)
    
    return g, g.n//g.batch_size

def present_results(m, image_dir, dg):
    """Given a model and information for image generation,
    make predictions and present the information."""
    g, steps = init_gen(image_dir, dg)
    Y_pred2 = m.predict_generator(g, steps=steps)
    y_pred2 = np.argmax(Y_pred2, axis=1)
    print('Confusion Matrix')
    print(confusion_matrix(g.classes, y_pred2))
    print('Classification Report')
    print(classification_report(g.classes, y_pred2, target_names=g.class_indices))
    print("F1 score (using average='micro')")
    print(f1_score(y_true=g.classes, y_pred=y_pred2, average='micro'))

# CNN From Nowka

In [4]:
# neural network model
# you may want to vary these parameters, etc

num_classes = 8 # fixed by the number of classes of signs that we gave you. Dont change

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam', # may want to try others
              metrics = ['accuracy'])
model.summary()

# Train and save.
train_gen, train_steps = init_gen(image_dir='train', dg=train_datagen)
val_gen, val_steps = init_gen(image_dir='validation', dg=valid_datagen)
model.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10 # may need to increase if not seeing low enough losses
)
# model.save('cnn_nowka.h5')
# del model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxP

# Part 2 Results

## Validation Data

In [5]:
# NOTE: Model has been saved, loading it and making predictions.
present_results(m=keras.models.load_model('cnn_nowka.h5'),
                image_dir='validation', dg=valid_datagen)

Found 929 images belonging to 8 classes.
Confusion Matrix
[[ 52   0   0   0   0   0   0   0]
 [  0  49   1   0   0   0   0   0]
 [  0   1 195   2   0   0   2   0]
 [  0   0   2 148   0   0   4   0]
 [  0   0   0   0  17   0   0   0]
 [  0   0   0   0   0  47   0   0]
 [  1   0   2   1   0   0 383   0]
 [  0   0   0   0   0   1   3  18]]
Classification Report
              precision    recall  f1-score   support

       merge       0.98      1.00      0.99        52
   keepRight       0.98      0.98      0.98        50
       yield       0.97      0.97      0.97       200
speedLimit35       0.98      0.96      0.97       154
speedLimit25       1.00      1.00      1.00        17
 signalAhead       0.98      1.00      0.99        47
  pedestrian       0.98      0.99      0.98       387
        stop       1.00      0.82      0.90        22

   micro avg       0.98      0.98      0.98       929
   macro avg       0.98      0.97      0.97       929
weighted avg       0.98      0.98      0.98

## Test Data

In [6]:
present_results(m=keras.models.load_model('cnn_nowka.h5'),
                image_dir='test', dg=test_datagen)

Found 884 images belonging to 8 classes.
Confusion Matrix
[[ 58   0   0   0   0   0   0   0]
 [  0  53   1   0   0   0   1   0]
 [  0   0 204   4   0   0   0   0]
 [  0   1   2 140   0   0   3   0]
 [  0   0   0   0  23   1   1   0]
 [  0   0   0   0   0  41   0   0]
 [  0   0   0   2   0   0 329   0]
 [  0   0   0   1   0   0   1  18]]
Classification Report
              precision    recall  f1-score   support

       merge       1.00      1.00      1.00        58
   keepRight       0.98      0.96      0.97        55
       yield       0.99      0.98      0.98       208
speedLimit35       0.95      0.96      0.96       146
speedLimit25       1.00      0.92      0.96        25
 signalAhead       0.98      1.00      0.99        41
  pedestrian       0.98      0.99      0.99       331
        stop       1.00      0.90      0.95        20

   micro avg       0.98      0.98      0.98       884
   macro avg       0.98      0.96      0.97       884
weighted avg       0.98      0.98      0.98

In [7]:
#This cell dumps out a file of which files were incorrectly predicted
#so you can see if you need more features, more training samples, etc
# predicted_class_indices=np.argmax(Y_pred,axis=1)
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]
# # NOTE/TODO: Change this to test_generator later.
# filenames=valid_generator.filenames
# # filenames=test_generator.filenames
# print(len(filenames))
# print(len(predictions))
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":predictions})
# results.to_csv("results.csv",index=False)

# Part 3

## Delete bad data from training

In [8]:
# The following are in 'yield' but are actually 'merge'
bad = [
    '140_yield_1323813350.avi_image2.png',
    '141_yield_1323813350.avi_image3.png',
    '142_yield_1323813350.avi_image4.png',
    '143_yield_1323813350.avi_image5.png',
    '144_yield_1323813350.avi_image6.png',
    '146_yield_1323813350.avi_image8.png',
    '192_yield_1323816786.avi_image1.png',
    '193_yield_1323816786.avi_image10.png',
    '194_yield_1323816786.avi_image11.png',
    '195_yield_1323816786.avi_image12.png',
    '196_yield_1323816786.avi_image13.png',
    '199_yield_1323816786.avi_image16.png',
    '204_yield_1323816786.avi_image20.png',
    '206_yield_1323816786.avi_image22.png',
    '207_yield_1323816786.avi_image23.png',
    '208_yield_1323816786.avi_image24.png',
    '209_yield_1323816786.avi_image25.png',
    '212_yield_1323816786.avi_image5.png',
    '213_yield_1323816786.avi_image6.png',
    '214_yield_1323816786.avi_image7.png',
    '215_yield_1323816786.avi_image8.png',
    '257_yield_1323821570.avi_image0.png',
    '258_yield_1323821570.avi_image1.png',
    '260_yield_1323821570.avi_image3.png',
    '261_yield_1323821570.avi_image4.png',
    '262_yield_1323821570.avi_image5.png',
    '264_yield_1323821570.avi_image7.png',
    '265_yield_1323821570.avi_image8.png',
]

train_dir = os.path.join(DATA_DIR, 'train')

for f in bad:
    os.rename(os.path.join(train_dir, 'yield', f), os.path.join(train_dir, 'merge', f))

## Recreate Image Data Generator(s)

In [9]:
# Use some rotations, shears, shifts.
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, shear_range=30,
                                   width_shift_range=0.25, height_shift_range=0.25, zoom_range=0.25)
                                   #horizontal_flip=True, vertical_flip=True)

# train_gen, train_steps = init_gen(image_dir='train', dg=train_datagen)
# val_gen, val_steps = init_gen(image_dir='validation', dg=valid_datagen)

# Our dataset is heavily imbalanced. Use class weights.
# NOTE: It turns out using weights really impacts training negatively.
# classes = train_gen.classes
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(classes), y=classes)
# weight_dict = {ind: val for ind, val in enumerate(class_weights)}
# print(weight_dict)

## Early Stopping

In [10]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,
                                          restore_best_weights=True)

## CNN 1 (deeper)

In [11]:
deep_file = 'cnn_deeper.h5' 

# Use more filters and one extra convolutional layer.
cnn = Sequential()
# Conv 1.
cnn.add(Conv2D(64, (3, 3), padding='same',input_shape = (32, 32, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
# Conv 2.
cnn.add(Conv2D(128, (3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
# Conv 3.
cnn.add(Conv2D(256, (2, 2), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
# Flatten.
cnn.add(Flatten())
# Dense.
cnn.add(Dense(512))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.25))
# Predict.
cnn.add(Dense(num_classes))
cnn.add(Activation('softmax'))

# Print summary, compile.
cnn.summary()
cnn.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

# Recreate generators to avoid annoying shifting issues.
train_gen, train_steps = init_gen(image_dir='train', dg=train_datagen)
val_gen, val_steps = init_gen(image_dir='validation', dg=valid_datagen)

# Train.
cnn.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=100,
                    callbacks=[early_stop]
                    #class_weight=weight_dict
)

# Save.
cnn.save(deep_file)

# Clear.
del cnn

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
activation_6 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 256)         131328    
__________

2744/2744 [==============================] - 20s 7ms/step - loss: 0.4098 - acc: 0.8670 - val_loss: 0.1973 - val_acc: 0.9386
Epoch 44/100
2744/2744 [==============================] - 20s 7ms/step - loss: 0.4034 - acc: 0.8695 - val_loss: 0.2801 - val_acc: 0.9117
Epoch 45/100
2744/2744 [==============================] - 20s 7ms/step - loss: 0.4306 - acc: 0.8728 - val_loss: 0.3910 - val_acc: 0.9064
Epoch 46/100
2744/2744 [==============================] - 20s 7ms/step - loss: 0.3835 - acc: 0.8779 - val_loss: 0.2291 - val_acc: 0.9429
Epoch 47/100
2744/2744 [==============================] - 20s 7ms/step - loss: 0.4262 - acc: 0.8703 - val_loss: 0.2282 - val_acc: 0.9300
Epoch 48/100
2744/2744 [==============================] - 20s 7ms/step - loss: 0.4012 - acc: 0.8772 - val_loss: 0.1829 - val_acc: 0.9365
Epoch 49/100
2744/2744 [==============================] - 20s 7ms/step - loss: 0.4174 - acc: 0.8728 - val_loss: 0.3015 - val_acc: 0.9268
Epoch 50/100
2744/2744 [==============================

## CNN 2 (shallower)

In [12]:
shallow_file = 'cnn_shallow.h5' 

# Use larger initial convolution layer with more filters.
cnn = Sequential()
# Conv 1.
cnn.add(Conv2D(128, (4, 4), padding='same',input_shape = (32, 32, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
# Conv 2.
cnn.add(Conv2D(256, (2, 2), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
# Flatten.
cnn.add(Flatten())
# Dense.
cnn.add(Dense(512))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.25))
# Predict.
cnn.add(Dense(num_classes))
cnn.add(Activation('softmax'))

# Print summary, compile.
cnn.summary()
cnn.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

# Recreate generators to avoid annoying shifting issues.
train_gen, train_steps = init_gen(image_dir='train', dg=train_datagen)
val_gen, val_steps = init_gen(image_dir='validation', dg=valid_datagen)

# Train.
cnn.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=100,
                    callbacks=[early_stop]
                    #class_weight=weight_dict
)

# Save.
cnn.save(shallow_file)

# Clear.
del cnn

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 128)       6272      
_________________________________________________________________
activation_10 (Activation)   (None, 32, 32, 128)       0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 256)       131328    
_________________________________________________________________
activation_11 (Activation)   (None, 16, 16, 256)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16384)             0         
__________

Epoch 46/100
2744/2744 [==============================] - 49s 18ms/step - loss: 0.4304 - acc: 0.8692 - val_loss: 0.2495 - val_acc: 0.9107
Epoch 47/100
2744/2744 [==============================] - 50s 18ms/step - loss: 0.4198 - acc: 0.8743 - val_loss: 0.3235 - val_acc: 0.9074
Epoch 48/100
2744/2744 [==============================] - 49s 18ms/step - loss: 0.4375 - acc: 0.8659 - val_loss: 0.2657 - val_acc: 0.9268
Epoch 49/100
2744/2744 [==============================] - 49s 18ms/step - loss: 0.3946 - acc: 0.8776 - val_loss: 0.2771 - val_acc: 0.9171
Epoch 50/100
2744/2744 [==============================] - 50s 18ms/step - loss: 0.4621 - acc: 0.8633 - val_loss: 0.2231 - val_acc: 0.9397


## CNN 3

In [13]:
nowka_file = 'cnn_nowka_mod.h5'

# Use Nowka's CNN, but use early stopping. Don't use class weights.
# Also note that we'll be using the new train_generator, which performs
# some image augmentation, and has the bad images moved.
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), padding='same',input_shape = (32, 32, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(64, (3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(512))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(num_classes))
cnn.add(Activation('softmax'))

cnn.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])
cnn.summary()

# Recreate generators to avoid annoying shifting issues.
train_gen, train_steps = init_gen(image_dir='train', dg=train_datagen)
val_gen, val_steps = init_gen(image_dir='validation', dg=valid_datagen)

# Train.
cnn.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=100,
                    callbacks=[early_stop]
                    #class_weight=weight_dict
)

# Save.
cnn.save(nowka_file)

# Clear.
del cnn

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_15 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4096)              0         
__________

2744/2744 [==============================] - 19s 7ms/step - loss: 0.4270 - acc: 0.8699 - val_loss: 0.2628 - val_acc: 0.9107
Epoch 47/100
2744/2744 [==============================] - 18s 7ms/step - loss: 0.4262 - acc: 0.8597 - val_loss: 0.2178 - val_acc: 0.9247
Epoch 48/100
2744/2744 [==============================] - 19s 7ms/step - loss: 0.4293 - acc: 0.8655 - val_loss: 0.2128 - val_acc: 0.9300
Epoch 49/100
2744/2744 [==============================] - 19s 7ms/step - loss: 0.4273 - acc: 0.8692 - val_loss: 0.2221 - val_acc: 0.9257
Epoch 50/100
2744/2744 [==============================] - 18s 7ms/step - loss: 0.4106 - acc: 0.8637 - val_loss: 0.2322 - val_acc: 0.9386
Epoch 51/100
2744/2744 [==============================] - 19s 7ms/step - loss: 0.3880 - acc: 0.8728 - val_loss: 0.1740 - val_acc: 0.9473
Epoch 52/100
2744/2744 [==============================] - 19s 7ms/step - loss: 0.4048 - acc: 0.8677 - val_loss: 0.2375 - val_acc: 0.9300
Epoch 53/100
2744/2744 [==============================

## CNN 4

In [18]:
no_aug = 'cnn_no_aug.h5'
train_datagen = ImageDataGenerator(rescale=1./255)
# Use Nowka's CNN, but with early stopping. Don't bother with image augmentation,
# as that seems to be leading to not as good results. This means that the data
# is likely pretty homogenous (which is true from looking at it), so the 
# robustness that comes from augmentation actually hurts us here.
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), padding='same',input_shape = (32, 32, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(64, (3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(512))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(num_classes))
cnn.add(Activation('softmax'))

cnn.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])
cnn.summary()

# Recreate generators to avoid annoying shifting issues.
train_gen, train_steps = init_gen(image_dir='train', dg=train_datagen)
val_gen, val_steps = init_gen(image_dir='validation', dg=valid_datagen)

# Train.
cnn.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=100,
                    callbacks=[early_stop]
                    #class_weight=weight_dict
)

# Save.
cnn.save(no_aug)

# Clear.
del cnn

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_19 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4096)              0         
__________

## Results

### Validation

In [19]:
# NOTE: models have been saved. Loading them up and making predictions.
print('Deeper:')
present_results(m=keras.models.load_model(deep_file),
                image_dir='validation', dg=valid_datagen)

print('*'*80)
print('Shallower:')
present_results(m=keras.models.load_model(shallow_file),
                image_dir='validation', dg=valid_datagen)

print('*'*80)
print('Modified Nowka:')
present_results(m=keras.models.load_model(nowka_file),
                image_dir='validation', dg=valid_datagen)

print('*'*80)
print('Modified Nowka, no image augmentation:')
present_results(m=keras.models.load_model(no_aug),
                image_dir='validation', dg=valid_datagen)

Deeper:
Found 929 images belonging to 8 classes.
Confusion Matrix
[[ 51   0   0   0   0   1   0   0]
 [  0  43   3   3   0   0   1   0]
 [  2   1 193   4   0   0   0   0]
 [  0   0   3 150   0   0   1   0]
 [  0   0   0   0  14   3   0   0]
 [  0   0   0   0   7  40   0   0]
 [  0   1   3   2   0   0 380   1]
 [  0   0   4   0   0   0   4  14]]
Classification Report
              precision    recall  f1-score   support

       merge       0.96      0.98      0.97        52
   keepRight       0.96      0.86      0.91        50
       yield       0.94      0.96      0.95       200
speedLimit35       0.94      0.97      0.96       154
speedLimit25       0.67      0.82      0.74        17
 signalAhead       0.91      0.85      0.88        47
  pedestrian       0.98      0.98      0.98       387
        stop       0.93      0.64      0.76        22

   micro avg       0.95      0.95      0.95       929
   macro avg       0.91      0.88      0.89       929
weighted avg       0.95      0.95  

### Testing

In [20]:
# NOTE: models have been saved. Loading them up and making predictions.
print('Deeper:')
present_results(m=keras.models.load_model(deep_file),
                image_dir='test', dg=test_datagen)
print('*'*80)
print('Shallower:')
present_results(m=keras.models.load_model(shallow_file),
                image_dir='test', dg=test_datagen)

print('*'*80)
print('Modified Nowka:')
present_results(m=keras.models.load_model(nowka_file),
                image_dir='test', dg=test_datagen)

print('*'*80)
print('Modified Nowka, no image augmentation:')
present_results(m=keras.models.load_model(no_aug),
                image_dir='test', dg=test_datagen)

print('*'*80)
print('Original Nowka:')
present_results(m=keras.models.load_model('cnn_nowka.h5'),
                image_dir='test', dg=test_datagen)



Deeper:
Found 884 images belonging to 8 classes.
Confusion Matrix
[[ 57   0   0   1   0   0   0   0]
 [  0  42   8   4   0   0   1   0]
 [  1   0 202   5   0   0   0   0]
 [  0   0   1 142   0   0   3   0]
 [  0   0   0   0  20   5   0   0]
 [  0   0   0   0   6  35   0   0]
 [  0   0   1   1   0   0 329   0]
 [  0   0   2   2   0   0   3  13]]
Classification Report
              precision    recall  f1-score   support

       merge       0.98      0.98      0.98        58
   keepRight       1.00      0.76      0.87        55
       yield       0.94      0.97      0.96       208
speedLimit35       0.92      0.97      0.94       146
speedLimit25       0.77      0.80      0.78        25
 signalAhead       0.88      0.85      0.86        41
  pedestrian       0.98      0.99      0.99       331
        stop       1.00      0.65      0.79        20

   micro avg       0.95      0.95      0.95       884
   macro avg       0.93      0.87      0.90       884
weighted avg       0.95      0.95  

# Final Testing Results for Best Model

In [32]:
present_results(m=keras.models.load_model(no_aug), image_dir='test', dg=test_datagen)
print('*'*80)
print('The final model used has an identical architecture to the initial CNN')
print('which Professor Nowka provided. However, it was trained after fixing ')
print('some of the training data, which was mislabeled. Also, the early_stop')
print('callback was used to train for longer and select the weights which ')
print('performed best on the validation set. This model was trained strictly')
print('on the training set, and not on the validation or testing data.')
print('')
print('As is evident in this notebook, several different architectures and ')
print('training techniques were attempted. As the majority of these techniques')
print('are self-evident by examining the notebook, I won\'t go into detail here.')
print('I will note, however, that my class weighting and image augmentation ')
print('techniques did not work well. I believe this is indicative of the ')
print('validation and testing sets being simply too similar to the training ')
print('sets.')
print('')
print('It\'s also worth mentioning that a lot of effort went into getting this')
print('running on my laptop\'s GPU, an NVIDIA GeForce GTX 1050 Ti. I manually')
print('installed the NVIDIA drivers on Linux, installed some Docker utilities for')
print('utilizing the graphics card, and then finally ran this notebook in a ')
print('Docker container derived from an official Tensorflow Docker image.')

Found 884 images belonging to 8 classes.
Confusion Matrix
[[ 58   0   0   0   0   0   0   0]
 [  0  52   0   3   0   0   0   0]
 [  0   0 206   2   0   0   0   0]
 [  0   0   0 142   0   0   3   1]
 [  0   0   0   0  24   1   0   0]
 [  0   0   0   0   0  41   0   0]
 [  0   0   0   0   0   0 331   0]
 [  0   0   0   0   0   0   1  19]]
Classification Report
              precision    recall  f1-score   support

       merge       1.00      1.00      1.00        58
   keepRight       1.00      0.95      0.97        55
       yield       1.00      0.99      1.00       208
speedLimit35       0.97      0.97      0.97       146
speedLimit25       1.00      0.96      0.98        25
 signalAhead       0.98      1.00      0.99        41
  pedestrian       0.99      1.00      0.99       331
        stop       0.95      0.95      0.95        20

   micro avg       0.99      0.99      0.99       884
   macro avg       0.99      0.98      0.98       884
weighted avg       0.99      0.99      0.99